# Elo for csgo teams

In [27]:
import pandas as pd
import math

In [28]:
match_history  = pd.read_csv('data/matches.csv',sep = ';')
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [29]:
match_history.head()

,Date,Team1,Team2,Map,Event,event_type,event_tier
0,22/12/19,Vitality (16),MOUZ (9),nuke Nuke,EPICENTER 2019,BigEvents,S
1,22/12/19,Vitality (16),MOUZ (12),mrg Mirage,EPICENTER 2019,BigEvents,S
2,22/12/19,MOUZ (19),Vitality (16),inf Inferno,EPICENTER 2019,BigEvents,S
3,22/12/19,MOUZ (16),Evil Geniuses (14),ovp Overpass,EPICENTER 2019,BigEvents,S
4,22/12/19,MOUZ (16),Evil Geniuses (12),trn Train,EPICENTER 2019,BigEvents,S


## Clean data

In [30]:
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
match_history['map'] = match_history['map'].str[4:]

### Removing duplicates

In [63]:
match_history.drop_duplicates(subset=match_history.columns.difference(['event_type']),keep='last',inplace=True)

## Change type

### Score

In [31]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [32]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [33]:
match_history['Winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [34]:
match_history['elo1'] = None
match_history['elo2'] = None

In [39]:
match_history

,date,team1,team2,map,event,event_type,event_tier,s1,s2,Winner,elo1,elo2
0,2019-12-22,Vitality,MOUZ,Nuke,EPICENTER 2019,BigEvents,S,16,9,True,None,None
1,2019-12-22,Vitality,MOUZ,Mirage,EPICENTER 2019,BigEvents,S,16,12,True,None,None
2,2019-12-22,MOUZ,Vitality,Inferno,EPICENTER 2019,BigEvents,S,19,16,True,None,None
3,2019-12-22,MOUZ,Evil Geniuses,Overpass,EPICENTER 2019,BigEvents,S,16,14,True,None,None
4,2019-12-22,MOUZ,Evil Geniuses,Train,EPICENTER 2019,BigEvents,S,16,12,True,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1086,2019-02-20,AVANGAR,Liquid,Overpass,IEM Katowice 2019,Majors,S,12,16,False,None,None
1087,2019-02-20,Natus Vincere,G2,Inferno,IEM Katowice 2019,Majors,S,16,7,True,None,None
1088,2019-02-20,HellRaisers,FaZe,Train,IEM Katowice 2019,Majors,S,4,16,False,None,None
1089,2019-02-20,ENCE,Renegades,Inferno,IEM Katowice 2019,Majors,S,13,16,False,None,None


# Applying elo function

In [76]:
#def eloretriver(df,team,date):
    #retrive the last elo point from a data frame
#    try:
#      df[df[date]]



def Probwin(rating1,rating2):
    # Function to calculate the Probability
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

def EloRatingcalculator(Ra,Rb,k,d):
    # If team has no rating the starting rating is 1500(?change)
    if Ra is None:
        Ra = 1500
    elif Rb is None:
        Rb = 1500
    # Function to calculate Elo rating
    # K is a constant.
    # d determines whether Player A wins or Player B.
    # To calculate the Winning
    # Probability of Player B
    Pb = Probwin(Ra, Rb)
 
    # To calculate the Winning
    # Probability of Player A
    Pa = Probwin(Rb, Ra)
      # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + k * (1 - Pa)
        Rb = Rb + k * (0 - Pb)
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        Ra = Ra + k * (0 - Pa)
        Rb = Rb + k * (1 - Pb)
    print("Updated Ratings:-")
    print("Ra =", round(Ra, 1)," Rb =", round(Rb, 1))
    return Ra,Rb

In [40]:
match_history.sort_values(by=["date"]).reset_index(drop=True,inplace=True)

K-factor  	Used for players with ratings ...
k=40 for a player new to the rating list until the completion of events with a total of 30 games, and for all players until their 18th birthday, as long as their rating remains under 2300.
k=20 	for players who have always been rated under 2400.
k=10for players with any published rating of at least 2400 and at least 30 games played in previous events. Thereafter it remains permanently at 10.

Score after mathces,series or lan torunaments(greater difference for tournaments?? Performance Rating by tournament?)

elo starting at what?

elo for players?

In [45]:
Ra = None
Rb = 2000
K = 30
d = 1
EloRatingcalculator(Ra, Rb, K, d)

Updated Ratings:-
Ra = 1528.4  Rb = 1971.6


(1528.4027935439394, 1971.5972064560606)

In [73]:
if match_history[(match_history['date']=='2019-02-20')&(match_history['team1'] == 'BIG')].shape[0] > 1:
    return match_history[(match_history['date']=='2019-02-20')&(match_history['team1'] == 'BIG')].tail

True